In [5]:
import pandas as pd
import numpy as np
import pyarrow.parquet as parquet

In [6]:
#mat = np.ones((10,80))
#names = ['a']*10

def createDF(mat, names):
    columns = ['class_' + str(x) for x in range(mat.shape[1]) ]
    df = pd.DataFrame(mat, columns=columns, dtype='float32')
    df['fileName'] = names

    cols = df.columns.tolist()
    cols = [cols[-1]]+cols[:-1] # or whatever change you need
    df = df.reindex(columns=cols)

    return df

In [7]:
testPath = '/home/adeykin/projects/SNAHackaton/testFeatures.csv'
trainPath = '/home/adeykin/projects/SNAHackaton/trainFeatures2.csv'

In [8]:
def readFeatures(fileName):
    numLines = sum(1 for line in open(fileName, 'r'))
    f = open(fileName, 'r')

    print('Read file: ' + fileName)
    print('Number of lines: ' + str(numLines))

    mat = np.zeros((numLines, 80), dtype='float32')
    names = []
    
    for i, line in enumerate(f):
        arr = line.split(',')
        #print(arr)
        names.append(arr[0])
        w, h = arr[1], arr[2]
        classesNum = int((len(arr) - 3) / 6)
        #print(classesNum)
        classesId = map(int, arr[3:3+classesNum])
        classesScores = map(float, arr[3+classesNum+1:3+classesNum+1+classesNum])
        for cid, score in zip(classesId, classesScores):
            mat[i, cid] = max(mat[i, cid], score)
        #print(classesId)
        #print(classesScores)
        if not i % 50000:
            print(str(i) + '/' + str(numLines) + ' lines precessed')
            
    return mat, names

In [9]:
testMat, testNames = readFeatures(testPath)

Read file: /home/adeykin/projects/SNAHackaton/testFeatures.csv
Number of lines: 352505
0/352505 lines precessed
50000/352505 lines precessed
100000/352505 lines precessed
150000/352505 lines precessed
200000/352505 lines precessed
250000/352505 lines precessed
300000/352505 lines precessed
350000/352505 lines precessed


In [10]:
np.save('testMat.npy', testMat)

In [11]:
testDF = createDF(testMat, testNames)

In [12]:
testDF.to_pickle('testDF.pkl')

In [37]:
trainMat, trainNames = readFeatures(trainPath)

Read file: /home/adeykin/projects/SNAHackaton/trainFeatures2.csv
Number of lines: 1629184
0/1629184 lines precessed
50000/1629184 lines precessed
100000/1629184 lines precessed
150000/1629184 lines precessed
200000/1629184 lines precessed
250000/1629184 lines precessed
300000/1629184 lines precessed
350000/1629184 lines precessed
400000/1629184 lines precessed
450000/1629184 lines precessed
500000/1629184 lines precessed
550000/1629184 lines precessed
600000/1629184 lines precessed
650000/1629184 lines precessed
700000/1629184 lines precessed
750000/1629184 lines precessed
800000/1629184 lines precessed
850000/1629184 lines precessed
900000/1629184 lines precessed
950000/1629184 lines precessed
1000000/1629184 lines precessed
1050000/1629184 lines precessed
1100000/1629184 lines precessed
1150000/1629184 lines precessed
1200000/1629184 lines precessed
1250000/1629184 lines precessed
1300000/1629184 lines precessed
1350000/1629184 lines precessed
1400000/1629184 lines precessed
1450000/

In [38]:
print(trainMat.shape)
print(len(trainNames))

(1629184, 80)
1629184


In [39]:
np.save('trainMat.npy', trainMat)

In [40]:
trainDF = createDF(trainMat, trainNames)

In [41]:
trainDF.to_pickle('trainDF.pkl')

# Target

In [2]:
trainMetaPath = '/home/adeykin/projects/SNAHackaton/imagesTrain'

In [3]:
trainMeta = parquet.read_table(trainMetaPath, columns = ['ImageId','feedback']).to_pandas()

In [5]:
trainMeta['targetScore'] = trainMeta['feedback'].map(lambda x:  (1 if 'Liked' in x else 0) + (-1 if 'Disliked' in x else 0) )

In [8]:
print('All: ' + str(trainMeta.shape[0]))
print('positivs: ' + str((trainMeta['targetScore'] > 0).sum()))
print('negativs: ' + str((trainMeta['targetScore'] < 0).sum()))


All: 17570921
positivs: 3332271
negativs: 1470063


**Aggtegate:**

In [18]:
trainMeta = trainMeta.drop(columns=['feedback'])

In [47]:
trainMeta['ImageId'] = trainMeta['ImageId'].map(lambda x: x[0])

In [60]:
g = trainMeta.groupby('ImageId', as_index = False)

In [61]:
aggr = g['targetScore'].sum()

In [64]:
aggr.to_pickle('trainTargetDF.pkl')

# Merge target and features

In [3]:
df = pd.read_pickle('trainDF.pkl')
df.head()

,fileName,class_0,class_1,class_2,class_3,class_4,class_5,class_6,class_7,class_8,...,class_70,class_71,class_72,class_73,class_74,class_75,class_76,class_77,class_78,class_79
0,000001306cae19a89e295296b506bab7,0.069727,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,000006419b6c05ec9cf30ef466dd8fec,0.058808,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.693916
2,000007d3372fb4688f870cb150d551f8,0.058399,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,000010bc53c1c6c567c112b7a17227fa,0.022514,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,00001a68648da02922172a45b42f5e16,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [19]:
#df = df.rename( {'fileName': 'ImageId'},  axis='columns')
df.head()

,ImageId,class_0,class_1,class_2,class_3,class_4,class_5,class_6,class_7,class_8,...,class_70,class_71,class_72,class_73,class_74,class_75,class_76,class_77,class_78,class_79
0,000001306cae19a89e295296b506bab7,0.069727,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,000006419b6c05ec9cf30ef466dd8fec,0.058808,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.693916
2,000007d3372fb4688f870cb150d551f8,0.058399,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,000010bc53c1c6c567c112b7a17227fa,0.022514,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,00001a68648da02922172a45b42f5e16,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [18]:
df['ImageId'].dtype

dtype('O')

In [20]:
dfTarget['ImageId'].dtype

dtype('O')

In [5]:
dfTarget = pd.read_pickle('trainTargetDF.pkl')
dfTarget.head()

,ImageId,targetScore
0,000001306cae19a89e295296b506bab7,0
1,000006419b6c05ec9cf30ef466dd8fec,6
2,000007d3372fb4688f870cb150d551f8,0
3,000010bc53c1c6c567c112b7a17227fa,0
4,00001a68648da02922172a45b42f5e16,1


In [22]:
df = pd.merge(left=df, right=dfTarget, on='ImageId')
df.head()

,ImageId,class_0,class_1,class_2,class_3,class_4,class_5,class_6,class_7,class_8,...,class_71,class_72,class_73,class_74,class_75,class_76,class_77,class_78,class_79,targetScore
0,000001306cae19a89e295296b506bab7,0.069727,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0
1,000006419b6c05ec9cf30ef466dd8fec,0.058808,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.693916,6
2,000007d3372fb4688f870cb150d551f8,0.058399,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0
3,000010bc53c1c6c567c112b7a17227fa,0.022514,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0
4,00001a68648da02922172a45b42f5e16,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1


In [23]:
df.to_pickle('trainWithtargetDF.pkl')

# Analise features

In [27]:
X = df[df.columns[1:-1]].values

In [30]:
print(X.shape)
print(X.shape[0]*X.shape[1])

(1442263, 80)
115381040


In [29]:
print('non zero elements: ' + str(np.sum(X>0)))

non zero elements: 6286996


In [31]:
print('big enought elements: ' + str(np.sum(X>0.5)))

big enought elements: 1420420


In [33]:
print('Number of zero rows:' + str(np.all(X == 0, axis=1).sum()))

114199

In [36]:
print('Number of zero columns:' + str(np.all(X < 0.5, axis=0).sum()))

Number of zero columns:0
